In [3]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, handoff
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [5]:
load_dotenv(override=True)

True

## Instructions

In [6]:
# Sales-focused copywriter
instructions1 = """
You are a results-driven marketing copywriter.
Your goal is to create persuasive and compelling marketing content that drives conversions.
You focus on benefits, urgency, strong CTAs, and value propositions.
Your tone is professional, confident, and tailored to decision-makers.
Avoid fluff—emphasize why this product or service solves a key pain point.
"""

# Story-driven brand voice
instructions2 = """
You are a storytelling-focused brand voice specialist.
You craft engaging narratives that build emotional connections with the audience.
Each piece of content should tell a mini story, using relatable characters, conflicts, and outcomes to draw readers in.
Focus on brand personality and human resonance.
Tone should be authentic, warm, and memorable.
"""

# Funny/informal tone
instructions3 = """
You are a witty and informal social media marketer.
Your job is to create fun, cheeky, and casual content that grabs attention and entertains.
Use humor, pop culture references, emojis (when appropriate), and playful language to make posts shareable and relatable.
Avoid sounding too corporate—think memes, GIF-worthy lines, and light-hearted jokes, while still getting the message across.
"""

Marketing manager

In [49]:
instructions_marketing = """
You are a marketing manager. You use the tools to generate different versions of the marketing email.
Your job is to evaluate all outputs and choose the most emotionally engaging email for our new users.
Only pick one email — do not combine or modify outputs.
After selecting, send the chosen email using the send_email tool.
"""

In [8]:
content_writer_1 = Agent(
    name = "Sales-focused Copywriter",
    instructions = instructions1,
    model = "gpt-4o-mini"
)

content_writer_2 = Agent(
    name = "Story-driven Brand Voice",
    instructions = instructions2,
    model = "gpt-4o-mini"
)

content_writer_3 = Agent(
    name = "Funny/Informal Tone",
    instructions = instructions3,
    model = "gpt-4o-mini"
)

## Tools

In [44]:
# @function_tool
# def send_email(body: str):
#     """ Send out an email with the given body to all sales prospects """
#     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
#     from_email = Email("aungkpp.dev@gmail.com")  # Change to your verified sender
#     to_email = To("aungkpp.dev@gmail.com")  # Change to your recipient
#     content = Content("text/plain", body)
#     mail = Mail(from_email, to_email, "Marketing email", content).get()
#     response = sg.client.mail.send.post(request_body=mail)
#     return {"status": "success"}

In [45]:
#send_email

### Convert agents into tools

In [46]:
description = "Write an marketing email about MovieJoys subscription which is a movie website."

tool1 = content_writer_1.as_tool(tool_name = "content_agent_1" , tool_description=description)
tool2 = content_writer_2.as_tool(tool_name = "content_agent_2" , tool_description=description)
tool3 = content_writer_3.as_tool(tool_name = "content_agent_3" , tool_description=description)

In [47]:
# marketing_manager = Agent(name = "Marketing Manager", instructions = instructions,tools = tools, model = "gpt-4o-mini")
#
# message = "Write a marketing email about MovieJoys subscription addressed to new users."
#
# with trace("Marketing Manager"):
#     result = await Runner.run(marketing_manager, message)

## Second Agent for Converting to HTML(Handoff)

In [38]:
subject_instructions = """
You are expert at writing the most emotionally engaging email for our new users.
Your task is to write a subject line for a marketing email about MovieJoys subscription, which is a movie website, that is likely to get a reply. You will be given the email body. Focus on curiosity, clarity, and value to the recipient.
Keep it short, compelling, and avoid sounding spammy.
"""

html_instructions = """
You are an email designer skilled in converting plain text emails into professional HTML.
You will be given a plain text email (which may contain markdown).
Your task is to convert it into a visually appealing HTML email with proper formatting, layout, and styling.

Use:
- Relevant emojis and images where appropriate
- A clear header and structure
- A call-to-action (CTA) button
- Styling that aligns with the product being promoted

Make sure the email looks polished and ready to be sent to customers.
"""

In [39]:
subject_writer = Agent(name = "Email subject writer", instructions = subject_instructions, model = "gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name = "subject_writer" , tool_description='Write a subject for a marketing email')

html_converter = Agent(name = "HTML email body converter", instructions = html_instructions, model = "gpt-4o-mini")
html_tools = html_converter.as_tool(tool_name = "html_converter" , tool_description='Convert a text email body to an HTML email')

In [53]:
@function_tool
def send_HTML_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("aungkpp.dev@gmail.com")  # Change to your verified sender
    to_email = To("aungkpp.dev@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [54]:
tools = [subject_tool, html_tools, send_HTML_email]

In [55]:
instructions_email = "You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

email_agent = Agent(
    name = "Email agent",
    instructions = instructions_email,
    tools = tools,
    model = "gpt-4o-mini",
    handoff_description = "Convert an email into HTML and send it"
)


## Building Marketing manager agent

In [56]:
tools = [tool1, tool2, tool3]
handoff = [email_agent]

print(tools)
print(handoff)

[FunctionTool(name='content_agent_1', description='Write an marketing email about MovieJoys subscription which is a movie website.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'content_agent_1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x158e479c0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='content_agent_2', description='Write an marketing email about MovieJoys subscription which is a movie website.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'content_agent_2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x158ee91c0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='content_agent_3', descriptio

## Combining 2 Agent

In [57]:
marketing_manager = Agent(
    name = "Marketing Manager",
    instructions = instructions_marketing,
    tools = tools,
    handoffs = handoff,
    model = "gpt-4o-mini"
)

message = "Write a marketing email about MovieJoys subscription addressed to new users."

with trace("Marketing Manager"):
    result = await Runner.run(marketing_manager, message)

In [51]:
print(result)

RunResult:
- Last agent: Agent(name="Email agent", ...)
- Final output (str):
    The marketing email for MovieJoys has been successfully sent to new users! If you need anything else or further assistance, feel free to ask. 🎬✨
- 19 new item(s)
- 5 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
